In [16]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import torch.nn as nn

from torchvision.models import resnet18, ResNet18_Weights
from torchvision.models import vit_b_16, ViT_B_16_Weights 
import torch
import PIL

In [3]:
import numpy as np
from PIL import Image

DATASET_PATH = '/kaggle/input/dog-emotion/Dog Emotion/'

data = {'img': [], 'emotion': []}

for emotion in ['angry', 'happy', 'relaxed', 'sad']:
    for dirname, _, filenames in os.walk(DATASET_PATH + emotion + '/'):
        for filename in filenames:
            data['img'].append(os.path.join(dirname, filename))
            data['emotion'].append(emotion)

df = pd.DataFrame.from_dict(data)

In [4]:
df

,img,emotion
0,/kaggle/input/dog-emotion/Dog Emotion/angry/RD...,angry
1,/kaggle/input/dog-emotion/Dog Emotion/angry/GJ...,angry
2,/kaggle/input/dog-emotion/Dog Emotion/angry/GX...,angry
3,/kaggle/input/dog-emotion/Dog Emotion/angry/b3...,angry
4,/kaggle/input/dog-emotion/Dog Emotion/angry/Hf...,angry
...,...,...
3995,/kaggle/input/dog-emotion/Dog Emotion/sad/VNqe...,sad
3996,/kaggle/input/dog-emotion/Dog Emotion/sad/0Bnt...,sad
3997,/kaggle/input/dog-emotion/Dog Emotion/sad/aWoj...,sad
3998,/kaggle/input/dog-emotion/Dog Emotion/sad/8ihS...,sad


In [5]:
emotion_labels_map = {}

unique_emotions = df['emotion'].unique()
for i in range(len(unique_emotions)):
    emotion_labels_map[unique_emotions[i]] = i

In [6]:
df['emotion_label'] = df['emotion'].apply(lambda x: emotion_labels_map[x])
df

,img,emotion,emotion_label
0,/kaggle/input/dog-emotion/Dog Emotion/angry/RD...,angry,0
1,/kaggle/input/dog-emotion/Dog Emotion/angry/GJ...,angry,0
2,/kaggle/input/dog-emotion/Dog Emotion/angry/GX...,angry,0
3,/kaggle/input/dog-emotion/Dog Emotion/angry/b3...,angry,0
4,/kaggle/input/dog-emotion/Dog Emotion/angry/Hf...,angry,0
...,...,...,...
3995,/kaggle/input/dog-emotion/Dog Emotion/sad/VNqe...,sad,3
3996,/kaggle/input/dog-emotion/Dog Emotion/sad/0Bnt...,sad,3
3997,/kaggle/input/dog-emotion/Dog Emotion/sad/aWoj...,sad,3
3998,/kaggle/input/dog-emotion/Dog Emotion/sad/8ihS...,sad,3


In [7]:
from torch.utils.data import DataLoader, Dataset

class Dataset(Dataset):
    def __init__(self, data, transform):
        self.data = data
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        image = PIL.Image.open(self.data.loc[idx, "img"]).convert('RGB')
        image = self.transform(image)
        label = torch.tensor(self.data.loc[idx, "emotion_label"])
        return image, label

In [8]:
from torchvision.transforms import v2
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df["img"], df["emotion_label"], random_state=42)

X_train.index = np.arange(len(X_train))
y_train.index = np.arange(len(y_train))
X_test.index = np.arange(len(X_test))
y_test.index = np.arange(len(y_test))

In [35]:
transform_train = transform_test = v2.Compose([
    v2.Resize(size=(224, 224)),
    v2.PILToTensor(),
    v2.ToDtype(torch.float32, scale=True),
    v2.RandomResizedCrop(224),  # Random crop with resize to 224x224
    v2.RandomHorizontalFlip(p=0.5),  # Horizontal flip with 50% probability
    v2.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Random color adjustments
    v2.RandomRotation(7),
    v2.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

transform_test = v2.Compose([
    v2.Resize(size=(224, 224)),
    v2.PILToTensor(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [36]:
train_ds = Dataset(pd.concat([X_train, y_train], axis=1), transform_train)
test_ds = Dataset(pd.concat([X_test, y_test], axis=1), transform_test)


In [37]:
import torch.nn as nn
import torch.optim as optim

model = vit_b_16(weights=ViT_B_16_Weights.DEFAULT)
model.heads.head = nn.Linear(in_features=768, out_features=4)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

num_epochs = 10
batch_size = 32
learning_rate = 1e-5
weight_decay = 0.0

train_loader = DataLoader(train_ds, batch_size=batch_size, drop_last=True)
test_loader = DataLoader(test_ds, batch_size=batch_size)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for i, (images, labels) in enumerate(train_loader):
        
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        if (i+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], '
                  f'Loss: {running_loss/100:.4f}, '
                  f'Accuracy: {100 * correct/total:.2f}%')
            running_loss = 0.0
    
print('Training complete')
    

Epoch [1/10], Step [10/93], Loss: 0.1361, Accuracy: 30.62%
Epoch [1/10], Step [20/93], Loss: 0.1354, Accuracy: 30.78%
Epoch [1/10], Step [30/93], Loss: 0.1255, Accuracy: 35.52%
Epoch [1/10], Step [40/93], Loss: 0.1210, Accuracy: 38.52%
Epoch [1/10], Step [50/93], Loss: 0.1105, Accuracy: 42.12%
Epoch [1/10], Step [60/93], Loss: 0.1071, Accuracy: 45.00%
Epoch [1/10], Step [70/93], Loss: 0.1028, Accuracy: 47.14%
Epoch [1/10], Step [80/93], Loss: 0.0961, Accuracy: 49.02%
Epoch [1/10], Step [90/93], Loss: 0.0888, Accuracy: 51.32%
Epoch [2/10], Step [10/93], Loss: 0.0790, Accuracy: 69.38%
Epoch [2/10], Step [20/93], Loss: 0.0765, Accuracy: 69.38%
Epoch [2/10], Step [30/93], Loss: 0.0757, Accuracy: 69.90%
Epoch [2/10], Step [40/93], Loss: 0.0717, Accuracy: 70.78%
Epoch [2/10], Step [50/93], Loss: 0.0708, Accuracy: 71.19%
Epoch [2/10], Step [60/93], Loss: 0.0676, Accuracy: 71.82%
Epoch [2/10], Step [70/93], Loss: 0.0696, Accuracy: 72.05%
Epoch [2/10], Step [80/93], Loss: 0.0635, Accuracy: 72.6

In [38]:
model.eval()

predictions = []
true_labels = []

for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    predictions += predicted.cpu().tolist()
    true_labels += labels.cpu().tolist()

In [39]:
from sklearn.metrics import accuracy_score

In [40]:
accuracy_score(predictions, true_labels)

0.858

In [29]:
torch.save(model.state_dict(), 'animals_vit_weights.pt')

In [34]:
from IPython.display import FileLink
FileLink(r'animals_vit_weights.pt')

/kaggle/working/animals_vit_weights.pt